In [9]:
import sqlite3
# Connecting to the database file
sqlite_file = "../mydatabase.db"
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

In [2]:
#CREATE TABLE TwitterStockName(date TEXT, stockName TEXT, fav INTEGER, likes INTEGER, tweet TEXT, categories TEXT, sensitivity TEXT , UNIQUE(date, stockName) ON CONFLICT REPLACE);

In [3]:
import os
import glob
import csv
import numpy as np
import re

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import numpy as np
import pickle

#training Classification

def trainCategoriesClassifier():
    twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)

    text_clf = Pipeline([('vect', CountVectorizer()),('tfidf',TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)),])
    #text_clf = Pipeline([('vect', CountVectorizer()),('tfidf',TfidfTransformer()),('clf', MultinomialNB()),])

    #Bayes Naive Train Model
    text_clf.fit(twenty_train.data, twenty_train.target)  
    with open('./pickleObj/CatClassifier.pkl', 'wb') as f: 
        pickle.dump([text_clf], f)
    f.close()
    
def loadCategoriesClassifier():
    with open('./pickleObj/CatClassifier.pkl','rb') as f:  # Python 3: open(..., 'rb')
        text_clf = pickle.load(f)
    f.close()
    return text_clf

def getIntoDir(name):
    path = './%s' % name
    os.chdir(path)
def getOutDir():
    path = '..'
    os.chdir(path)
             
def loadTweetsFile():
    getIntoDir('TweetsData')
#     print (os.getcwd())
    extension = 'csv'
    result = [i for i in glob.glob('*.{}'.format(extension))]
    
    getOutDir()
    return result

def removeSubString(s,sub):
    start = s.find(sub)
    end = s.find(' ', start)
    if (end ==-1):
        end = len(s)
    tmp = s[:start] + s[end:]
    if (tmp.find(sub)!=-1):
        tmp = removeSubString(tmp,sub)
    return (tmp)

def readFile(fname):
    getIntoDir('TweetsData')
    ans = []
    with open(fname) as f:
        reader = csv.reader(f,delimiter=';')
        for row in reader:
            tmp = [row[0],row[1],row[2],row[3],row[4]]
            ans.append(tmp)
    getOutDir()
    return ans

def cleanString(s):
    tmp = removeSubString(s,"http")
    tmp = removeSubString(tmp,"@")
    tmp = removeSubString(tmp,"#")
    tmp = removeSubString(tmp,"&amp")
    tmp = re.sub('[^a-zA-Z ]+', ' ', tmp)
    tmp = re.sub( '\s+', ' ', tmp ).strip()
    return tmp.lower()

def getCategory(result):
    text_clf=loadCategoriesClassifier()[0]
#     print (text_clf)
    ans = []
    for f in result:
        ans = ans+readFile(f)
    data = np.array(ans)

    tweetL = data[:,4].tolist()
    tweetLClean=[]
    for i in tweetL:
        tmp = cleanString(i)
        tweetLClean.append(tmp)
    return data,text_clf.predict(tweetLClean)

def getData():
    twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
    data,predicted = getCategory(loadTweetsFile())
    cat=twenty_train.target_names
    tmp=[]
    for i in predicted:
        tmp.append(cat[i])
    predicted = tmp
    predictedNP=np.transpose(np.array(predicted))
    ans = np.column_stack((data, predictedNP))
    return ans
    
#trainCategoriesClassifier()

finalData = getData()



/home/ubun/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def insertToDatabase(data,c):
    #c is connector SQL
    table_name = "Twitter"
    name_f = "name"
    fav_f ="fav"
    like_f = "likes"
    cat_f = "categories"
    sen_f = "sensitivity"
    for d in data:
        c.execute("INSERT INTO {tn} VALUES(?,?,?,?,?,?)".format(tn=table_name),d)
                  
# In[10]:
insertToDatabase(finalData,c)

In [10]:

table_name = "Twitter"
c.execute("INSERT INTO {tn} VALUES(?,?,?,?,?,?)".format(tn=table_name),finalData[0])

In [14]:
conn.commit()

conn.close()